In [1]:
!pip install transformers
!pip install datasets
!pip install seqeval
!pip install ipywidgets
!pip install IProgress

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [2]:
import torch

In [3]:
import transformers
from datasets import load_dataset, load_metric
from datasets import Sequence, ClassLabel
from datasets import Value

In [4]:
task = "ner" # Should be one of "ner", "pos" or "chunk"
model_checkpoint = "bert-base-uncased"
batch_size = 16

In [5]:
# dataset = load_dataset("conll2003", revision="master")

In [6]:
# dataset

In [7]:
PE_DATA_FILE = '/notebooks/token_classification/data/pe_dataset_task_1_by_essay_new.pt'

In [8]:
pe_dataset = torch.load(PE_DATA_FILE)

In [9]:
pe_dataset

DatasetDict({
    train: Dataset({
        features: ['essay_nr', 'split', 'essay', 'essay_words', 'essay_multi_class_tags', 'essay_multi_class_tags_int', 'essay_bio_only_tags', 'essay_bio_only_tags_int'],
        num_rows: 249
    })
    test: Dataset({
        features: ['essay_nr', 'split', 'essay', 'essay_words', 'essay_multi_class_tags', 'essay_multi_class_tags_int', 'essay_bio_only_tags', 'essay_bio_only_tags_int'],
        num_rows: 79
    })
    validation: Dataset({
        features: ['essay_nr', 'split', 'essay', 'essay_words', 'essay_multi_class_tags', 'essay_multi_class_tags_int', 'essay_bio_only_tags', 'essay_bio_only_tags_int'],
        num_rows: 63
    })
})

In [10]:
pe_dataset['train'].features

{'essay_nr': Value(dtype='string', id=None),
 'split': Value(dtype='string', id=None),
 'essay': Value(dtype='string', id=None),
 'essay_words': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'essay_multi_class_tags': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'essay_multi_class_tags_int': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None),
 'essay_bio_only_tags': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'essay_bio_only_tags_int': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None)}

### my work today

In [11]:
# what if we sequence the int labels to the class labels? will it work?

In [12]:
pe_dataset['train'].features['essay_bio_only_tags_int'] = Sequence(feature=ClassLabel(num_classes=3, names=['O', 'B-COMP', 'I-COMP'], names_file=None, id=None),
                                             length=-1, id=None)

In [13]:
pe_dataset['train'].features['essay_bio_only_tags_int']

Sequence(feature=ClassLabel(num_classes=3, names=['O', 'B-COMP', 'I-COMP'], names_file=None, id=None), length=-1, id=None)

In [14]:
pe_dataset['test'].features['essay_bio_only_tags_int'] = Sequence(feature=ClassLabel(num_classes=3, names=['O', 'B-COMP', 'I-COMP'], names_file=None, id=None),
                                             length=-1, id=None)

In [15]:
pe_dataset['validation'].features['essay_bio_only_tags_int'] = Sequence(feature=ClassLabel(num_classes=3, names=['O', 'B-COMP', 'I-COMP'], names_file=None, id=None),
                                             length=-1, id=None)

In [16]:
pe_dataset['test'].features['essay_bio_only_tags_int']

Sequence(feature=ClassLabel(num_classes=3, names=['O', 'B-COMP', 'I-COMP'], names_file=None, id=None), length=-1, id=None)

In [17]:
pe_dataset['validation'].features['essay_bio_only_tags_int']

Sequence(feature=ClassLabel(num_classes=3, names=['O', 'B-COMP', 'I-COMP'], names_file=None, id=None), length=-1, id=None)

In [18]:
pe_dataset['train'].features

{'essay_nr': Value(dtype='string', id=None),
 'split': Value(dtype='string', id=None),
 'essay': Value(dtype='string', id=None),
 'essay_words': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'essay_multi_class_tags': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'essay_multi_class_tags_int': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None),
 'essay_bio_only_tags': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'essay_bio_only_tags_int': Sequence(feature=ClassLabel(num_classes=3, names=['O', 'B-COMP', 'I-COMP'], names_file=None, id=None), length=-1, id=None)}

In [19]:
from transformers import AutoTokenizer

In [20]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [21]:
def align_labels_with_tokens(labels, word_ids):
    new_labels = []
    current_word = None
    for word_id in word_ids:
        if word_id != current_word:
            # Start of a new word!
            current_word = word_id
            label = -100 if word_id is None else labels[word_id]
            new_labels.append(label)
        elif word_id is None:
            # Special token
            new_labels.append(-100)
        else:
            # Same word as previous token
            label = labels[word_id]
            # If the label is B-XXX we change it to I-XXX
            if label % 2 == 1:
                label += 1
            new_labels.append(label)

    return new_labels

In [22]:
def tokenize_and_align_labels_hf(examples):
    tokenized_inputs = tokenizer(
        examples["essay_words"], truncation=True, is_split_into_words=True
    )
    all_labels = examples["essay_bio_only_tags_int"]
    #print(examples["paragraph_tags"])
    #all_labels = labels_object.str2int(examples["paragraph_tags"])
    
    new_labels = []
    for i, labels in enumerate(all_labels):
        word_ids = tokenized_inputs.word_ids(i)
        new_labels.append(align_labels_with_tokens(labels, word_ids))

    tokenized_inputs["labels"] = new_labels
    return tokenized_inputs

In [23]:
label_all_tokens = True

In [24]:
tokenized_datasets = pe_dataset.map(tokenize_and_align_labels_hf, batched=True)

Parameter 'function'=<function tokenize_and_align_labels_hf at 0x7f67b19b5280> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [25]:
label_list = pe_dataset["train"].features['essay_bio_only_tags_int'].feature.names

In [26]:
label_list

['O', 'B-COMP', 'I-COMP']

In [27]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list))

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForTokenClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-u

In [28]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwis

In [29]:
model_name = model_checkpoint.split("/")[-1]
args = TrainingArguments(
    f"{model_name}-finetuned-{task}",
    evaluation_strategy = "epoch",
    learning_rate=1e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=48,
    weight_decay=0.01,
    eval_steps=20,
    logging_steps=20
    #push_to_hub=True,
)

In [30]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

In [31]:
metric = load_metric("seqeval")

In [32]:
import numpy as np

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [33]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [34]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: essay_nr, split, essay, essay_multi_class_tags_int, essay_bio_only_tags, essay_multi_class_tags, essay_bio_only_tags_int, essay_words.
***** Running training *****
  Num examples = 249
  Num Epochs = 48
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 768


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.563756,0.255656,0.491839,0.336435,0.815549
2,0.739700,0.419324,0.396327,0.563656,0.465409,0.853625
3,0.449900,0.353050,0.483178,0.562568,0.519859,0.864204
4,0.342900,0.307968,0.563571,0.680087,0.616371,0.893297
5,0.297400,0.304393,0.508591,0.644178,0.568411,0.894938
6,0.297400,0.265824,0.544915,0.699674,0.612673,0.904286
7,0.255600,0.258234,0.534790,0.719260,0.613457,0.906293
8,0.229200,0.259337,0.531625,0.722524,0.612546,0.908618
9,0.192600,0.239015,0.573702,0.745375,0.648367,0.915914
10,0.172400,0.252012,0.604078,0.773667,0.678435,0.914227


The following columns in the evaluation set  don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: essay_nr, split, essay, essay_multi_class_tags_int, essay_bio_only_tags, essay_multi_class_tags, essay_bio_only_tags_int, essay_words.
***** Running Evaluation *****
  Num examples = 63
  Batch size = 16
The following columns in the evaluation set  don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: essay_nr, split, essay, essay_multi_class_tags_int, essay_bio_only_tags, essay_multi_class_tags, essay_bio_only_tags_int, essay_words.
***** Running Evaluation *****
  Num examples = 63
  Batch size = 16
The following columns in the evaluation set  don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: essay_nr, split, essay, essay_multi_class_tags_int, essay_bio_only_tags, essay_multi_class_tags, essay_bio_only_tags_int, essay_words.
***** Running Evaluation 

TrainOutput(global_step=768, training_loss=0.11051643698010594, metrics={'train_runtime': 667.7361, 'train_samples_per_second': 17.899, 'train_steps_per_second': 1.15, 'total_flos': 2980600775381178.0, 'train_loss': 0.11051643698010594, 'epoch': 48.0})

In [35]:
trainer.evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: essay_nr, split, essay, essay_multi_class_tags_int, essay_bio_only_tags, essay_multi_class_tags, essay_bio_only_tags_int, essay_words.
***** Running Evaluation *****
  Num examples = 63
  Batch size = 16


{'eval_loss': 0.3695220351219177,
 'eval_precision': 0.6572727272727272,
 'eval_recall': 0.7867247007616975,
 'eval_f1': 0.7161961367013372,
 'eval_accuracy': 0.9207934336525307,
 'eval_runtime': 1.5322,
 'eval_samples_per_second': 41.119,
 'eval_steps_per_second': 2.611,
 'epoch': 48.0}

In [36]:
predictions, labels, _ = trainer.predict(tokenized_datasets["test"])
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
results

The following columns in the test set  don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: essay_nr, split, essay, essay_multi_class_tags_int, essay_bio_only_tags, essay_multi_class_tags, essay_bio_only_tags_int, essay_words.
***** Running Prediction *****
  Num examples = 79
  Batch size = 16


{'COMP': {'precision': 0.728500355366027,
  'recall': 0.8154335719968179,
  'f1': 0.7695195195195196,
  'number': 1257},
 'overall_precision': 0.728500355366027,
 'overall_recall': 0.8154335719968179,
 'overall_f1': 0.7695195195195196,
 'overall_accuracy': 0.928154124356656}

In [37]:
# task: try to do only with the BIO. then see the results.

In [38]:
predictions, labels, _ = trainer.predict(tokenized_datasets["train"])
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results_train = metric.compute(predictions=true_predictions, references=true_labels)
results_train

The following columns in the test set  don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: essay_nr, split, essay, essay_multi_class_tags_int, essay_bio_only_tags, essay_multi_class_tags, essay_bio_only_tags_int, essay_words.
***** Running Prediction *****
  Num examples = 249
  Batch size = 16


{'COMP': {'precision': 0.9588969823100937,
  'recall': 0.9753903148981212,
  'f1': 0.9670733307096944,
  'number': 3779},
 'overall_precision': 0.9588969823100937,
 'overall_recall': 0.9753903148981212,
 'overall_f1': 0.9670733307096944,
 'overall_accuracy': 0.997657870669849}